# 🎨 AI Pictionary - CNN Model Training (Colab Version)

**Quick Draw Dataset Classification with TensorFlow/Keras**

Ce notebook est adapté pour tourner sur **Google Colab**. Il télécharge automatiquement les données brutes depuis les serveurs de Google, les traite, entraîne le modèle et vous permet de télécharger le fichier `.h5` final.

---

In [ ]:
# @title ⚙️ 1. Configuration de l'environnement Colab
import os

# Création de l'arborescence de dossiers nécessaire
os.makedirs("logs", exist_ok=True)
os.makedirs("backend/models", exist_ok=True)
os.makedirs("data", exist_ok=True)

print("✅ Dossiers créés : logs/, backend/models/, data/")

In [ ]:
# @title 2️⃣ Import des Librairies
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import os
import urllib.request
from datetime import datetime

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# @title 3️⃣ Téléchargement et Préparation des Données
# Cette cellule remplace le chargement HDF5 local.
# Elle télécharge les fichiers .npy directement depuis Google Cloud.

# Liste de 20 catégories pour la démo
CATEGORIES = [
    "apple", "banana", "baseball", "book", "bucket",
    "camera", "car", "clock", "cloud", "cup",
    "door", "eye", "face", "fan", "flower",
    "ladder", "lightning", "star", "sword", "tree"
]
NUM_CLASSES = len(CATEGORIES)
MAX_SAMPLES_PER_CLASS = 12000 # Limite pour éviter de saturer la RAM Colab

print(f"📥 Téléchargement des données pour {NUM_CLASSES} catégories...")
base_url = "https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/"

X_data = []
y_data = []

for idx, category in enumerate(CATEGORIES):
    # Gestion des espaces dans les noms de fichiers url (ex: ice cream)
    filename = category.replace(" ", "%20") + ".npy"
    url = base_url + filename
    local_path = f"data/{category}.npy"
    
    if not os.path.exists(local_path):
        try:
            urllib.request.urlretrieve(url, local_path)
        except Exception as e:
            print(f"❌ Erreur téléchargement {category}: {e}")
            continue
    
    # Charger les données
    data = np.load(local_path)
    # On garde seulement un sous-ensemble pour la mémoire
    data = data[:MAX_SAMPLES_PER_CLASS] 
    
    X_data.append(data)
    y_data.append(np.full(data.shape[0], idx))
    print(f"   ✅ {category}: {data.shape[0]} images chargées")

# Concaténation
X = np.concatenate(X_data, axis=0)
y = np.concatenate(y_data, axis=0)

# Normalisation [0, 1] et Reshape (N, 28, 28, 1)
print("\n🔄 Normalisation et Reshape...")
X = X.astype('float32') / 255.0
X = X.reshape(-1, 28, 28, 1)

# Split Train/Val/Test (80% / 10% / 10%)
print("✂️  Splitting dataset...")
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

IMAGE_SHAPE = (28, 28, 1)

# One-hot encoding
y_train_cat = to_categorical(y_train, NUM_CLASSES)
y_val_cat = to_categorical(y_val, NUM_CLASSES)
y_test_cat = to_categorical(y_test, NUM_CLASSES)

print(f"\n✅ Dataset prêt !")
print(f"   Train: {X_train.shape} - Label shape: {y_train_cat.shape}")
print(f"   Val:   {X_val.shape}")
print(f"   Test:  {X_test.shape}")

In [ ]:
# @title 4️⃣ Visualisation des échantillons
# Display random samples (one per category)
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
fig.suptitle('Sample Images from Each Category', fontsize=16, fontweight='bold')

for i, category in enumerate(CATEGORIES):
    if i >= 20: break # Safety check
    # Find first image of this category in train set
    idx = np.where(y_train == i)[0][0]
    
    # Plot image
    ax = axes[i // 5, i % 5]
    ax.imshow(X_train[idx].squeeze(), cmap='gray')
    ax.set_title(category, fontsize=12, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# @title 5️⃣ Architecture CNN Simple
# Build model
model = keras.Sequential([
    layers.Input(shape=IMAGE_SHAPE),
    
    # Conv Block 1
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", name="conv2d_1"),
    layers.MaxPooling2D(pool_size=(2, 2), name="maxpool_1"),
    
    # Conv Block 2
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", name="conv2d_2"),
    layers.MaxPooling2D(pool_size=(2, 2), name="maxpool_2"),
    
    # Head
    layers.Flatten(name="flatten"),
    layers.Dropout(0.5, name="dropout"),
    layers.Dense(NUM_CLASSES, activation="softmax", name="output")
], name="QuickDraw_SimpleCNN")

model.summary()

# Visualize model architecture (Chemin adapté pour Colab)
keras.utils.plot_model(
    model,
    to_file="logs/model_architecture.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    dpi=150
)
print("✅ Architecture sauvegardée dans logs/model_architecture.png")

In [ ]:
# @title 6️⃣ Compilation
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)
print("✅ Modèle compilé avec Adam & Categorical Crossentropy")

In [ ]:
# @title 7️⃣ Entraînement
BATCH_SIZE = 128
EPOCHS = 15

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=3, restore_best_weights=True, verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='logs/best_model.h5', # Chemin adapté
        monitor='val_accuracy', save_best_only=True, verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001, verbose=1
    )
]

print(f"🚀 Démarrage de l'entraînement sur {len(X_train):,} images...")
history = model.fit(
    X_train, y_train_cat,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val_cat),
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# @title 8️⃣ Historique d'entraînement
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train Acc')
axes[0].plot(history.history['val_accuracy'], label='Val Acc')
axes[0].set_title('Model Accuracy')
axes[0].legend()

# Loss
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Val Loss')
axes[1].set_title('Model Loss')
axes[1].legend()

plt.savefig('logs/training_history.png')
plt.show()

In [ ]:
# @title 9️⃣ Évaluation sur le Test Set
test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)

print(f"\n🎯 Test Accuracy: {test_accuracy*100:.2f}%")
print(f"🎯 Test Loss:     {test_loss:.4f}")

In [ ]:
# @title 🔟 Matrice de Confusion
y_pred_probs = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('logs/confusion_matrix.png')
plt.show()

In [ ]:
# @title 1️⃣1️⃣ Sauvegarde et Métadonnées
import json

MODEL_VERSION = "v1.0.0"
# Chemins adaptés pour Colab
MODEL_SAVE_PATH = f"backend/models/quickdraw_{MODEL_VERSION}.h5"
METADATA_PATH = f"backend/models/quickdraw_{MODEL_VERSION}_metadata.json"

# Save Model
model.save(MODEL_SAVE_PATH)
print(f"✅ Modèle sauvegardé : {MODEL_SAVE_PATH}")

# Save Metadata
metadata = {
    "version": MODEL_VERSION,
    "created_at": datetime.now().isoformat(),
    "test_accuracy": float(test_accuracy),
    "categories": CATEGORIES,
    "num_classes": NUM_CLASSES
}

with open(METADATA_PATH, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Métadonnées sauvegardées : {METADATA_PATH}")

In [ ]:
# @title ⬇️ 1️⃣2️⃣ Télécharger le modèle sur votre PC
from google.colab import files

print("Préparation du téléchargement...")
try:
    files.download(MODEL_SAVE_PATH)
    files.download(METADATA_PATH)
    print("✅ Téléchargement lancé dans le navigateur !")
except Exception as e:
    print(f"❌ Erreur lors du téléchargement: {e}")